In [ ]:
#!/usr/bin/env python
###################################
#                                 #
# File coded by: Robert J. Koch   #
#                                 #
###################################
#
# Tutorial 3 Refining PDFs from SrFe2As2 to investigate phase transformation
#
# This Diffpy-CMI script will carry out a structural refinement of a measured
# PDF from SrFe2As2.
#
# Import packages that we will need.
from pathlib import Path
import yaml
import pyjokes
import sys
sys.path.append(str(Path().absolute().parent.parent.parent))

from diffpy.srfit.fitbase import FitResults
from diffpy.structure.parsers import getParser
from scipy.optimize import least_squares
from cmi_demos.utils.helpers import makerecipe
from cmi_demos.utils.helpers import plotresults
print(pyjokes.get_joke())

############### Config ##############################
# Give a file path to where your pdf (.gr) and (.cif) files are located.
DPATH = Path("data")

# Give an identifying name for the refinement.
# Here this is just part of the basename we will use,
# as we will consider different structure models and
# different temperatures.
FIT_ID_BASE = "Fit_SrFe2As2_"

# Specify a string pattern for the PDF and cif files.
# Again, this is just part of the filename we will use,
# as we will consider different structure models and
# different temperatures.
# Each of our *.cif files and *.gr files we want to use will need
# to match this pattern.
GR_NAME_BASE = "SrFe2As2_"
CIF_NAME_BASE = GR_NAME_BASE

# This will allow us to skip things we've already done
SKIP_DONE = False

######## Experimental PDF Config ######################
# Specify the min, max, and step r-values of the PDF (that we want to fit over)
# also, specify the Q_max and Q_min values used to reduce the PDF.
PDF_RMIN = 1.5
PDF_RMAX = 50
PDF_RSTEP = 0.01
QMAX = 25
QMIN = 0.1

########PDF initialize refinable variables #############
# In this case, initial values for the lattice
# parameters and ADPs will be taken directly from the .cif
# structures, so we don't need to specify them here.
SCALE_I = 0.4
DELTA1_I = 1.6

# Instrumental will be fixed based on values obtained from a
# separate calibration step. These are hard-coded here.
QDAMP_I = 0.0349
QBROAD_I = 0.0176

In [ ]:

    # Make some folders to store our output files.
    resdir = "res"
    fitdir = "fit"
    figdir = "fig"

    folders = [resdir, fitdir, figdir]

    # Let's define our working directory.
    base_dir = Path()

    yaml_file = base_dir / FIT_ID_BASE / "refined_params.yml"

    # This is a bit different than what we've done before.
    # We are going to look at a set of temperatures, so we want
    # to find all the relevant data files in the "DPATH" folder
    # we identified earlier which match a certain pattern.

    # To do this we will use list comprehension

    # For every file we find, we will link it up with the data path
    data_files = list(DPATH.glob(f"*{GR_NAME_BASE}*.gr"))

                  # We look at all the files in the "DPATH" directory...

                 # ... that contain the string in "GR_NAME_BASE"

    # We now want to grab the temperature at which each file was measured.
    # We again use list comprehension, and we re-use the variable "temp"
    # This specific procedure depends on how the file is named.
    # In our case, we carefully named each file as:
    # "composition_TTTK.gr" where TTT is the temperature in Kelvin.

    # First we strip off the directory information for every file in "data_files"
    # This gives us a list of just full file names, without directories.
    temps = [f.stem for f in data_files]

    # Next we split every base filename into a list, delimited by "_"
    # We keep the second entry from this list, because we know it has the
    # temperature in the form TTTK.
    temps = [t.split('_')[1] for t in temps]

    # We want the temperature as an integer, so we need to drop the "K"
    # from each string and cast the values as integers using "int()"
    # Strings can be slides like arrays, so "[:-1]" means "take all the
    # values except the last one."
    temps = [int(t[:-1]) for t in temps]

    # This will sort the data files and temperatures in descending order
    # based on the temperature values.
    temps, data_files = zip(*sorted(zip(temps, data_files), reverse=True))

    # We want to test two structure models, so we should find both the cif files.
    # Similar to how we found the data files, we use list comprehension

    # For every file we find, we will link it up with the data path
    cif_files = list(DPATH.glob(f"*{CIF_NAME_BASE}*.cif"))

                # We look at all the files in the "DPATH" directory.

                # We only consider files that end with ".cif" ...

                 # ... and that contain the string in "CIF_NAME_BASE"

    # We initialize and empty dictionary, where we will save all
    # the details of the refined parameters.
    refined_dict = dict()

